# 01 - Data Loading

This notebook loads the NeRF Synthetic dataset and prepares it for training.

In [ ]:
# === Cell 2: load dataset (synthetic) ===
import os, json
from pathlib import Path
import numpy as np
import imageio.v2 as imageio

# For local use, change this path
DATA_DIR = Path("data/nerf_synthetic/lego")  # UPDATE THIS PATH
DEPTH_DIR = Path("data/nerf_synthetic/lego")  # Path to depth .npy files

def jload(p):
    with open(p, "r") as f:
        return json.load(f)

def load_synthetic_split(split="train", half_res=False):
    meta = jload(DATA_DIR / f"transforms_{split}.json")
    
    # Load depth npy if available
    npy_depth_path = DEPTH_DIR / f"lego_depths_{split}.npy"
    if npy_depth_path.exists():
        print(f"Loading depth supervision from {npy_depth_path}")
        depth_npy = np.load(npy_depth_path)
    else:
        depth_npy = None
    
    imgs, poses, depths = [], [], []
    
    for i, frame in enumerate(meta["frames"]):
        # Load image
        img_path = DATA_DIR / frame["file_path"]
        if not img_path.suffix:
            img_path = img_path.with_suffix(".png")
        img = imageio.imread(img_path)
        
        if img.dtype == np.uint16:
            img = (img / 65535.0).astype(np.float32)
        else:
            img = (img / 255.0).astype(np.float32)
        
        if half_res:
            img = img[::2, ::2]
        imgs.append(img)
        
        # Pose
        poses.append(np.array(frame["transform_matrix"], dtype=np.float32))
        
        # Depth from .npy
        if depth_npy is not None:
            d = depth_npy[i]
            if half_res:
                d = d[::2, ::2]
            depths.append(d)
        else:
            depths.append(None)
    
    imgs = np.stack(imgs, 0)
    if imgs.shape[-1] == 4:
        imgs = imgs[..., :3]
    
    poses = np.stack(poses, 0)
    
    if any(d is not None for d in depths):
        depths = np.stack(depths, 0)
    else:
        depths = None
    
    H, W = imgs.shape[1:3]
    camera_angle_x = meta["camera_angle_x"]
    focal = 0.5 * W / np.tan(0.5 * camera_angle_x)
    
    return imgs, poses, depths, H, W, focal

# Load training and validation splits
imgs_train, poses_train, depths_train, H, W, focal = load_synthetic_split("train", half_res=True)
imgs_val, poses_val, *_ = load_synthetic_split("val", half_res=True)

print(f"Train images: {imgs_train.shape}")
print(f"Train poses: {poses_train.shape}")
if depths_train is not None:
    print(f"Train depths: {depths_train.shape}")
    print(f"Depth range: [{depths_train.min():.2f}, {depths_train.max():.2f}]")
print(f"H={H}, W={W}, focal={focal:.2f}")